Extractions Silver

In [1]:
import os 
import sys
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
# Importar modulos propios en notebook
project_root = Path(os.getcwd()).parent
sys.path.append(str(project_root))

In [3]:
import dependencies as dp

In [4]:
from processes.extract.bronze import historic_df_bronze, resent_df_bronze
from db.engines import engine_bronze, conn_bronze

2024-12-03 19:45:13,030 - INFO - Successful connection to schema bronze
2024-12-03 19:45:13,064 - INFO - Successful connection to schema silver
2024-12-03 19:45:13,095 - INFO - Successful connection to schema gold


:D


In [5]:
schema = 'bronze'

In [6]:
bronze_df = resent_df_bronze(engine=engine_bronze, schema=schema)

2024-12-03 19:45:13,119 - INFO - Mapping database tables in schema 'bronze' started.
2024-12-03 19:45:13,203 - INFO - Tables in schema 'bronze' were successfully mapped.


In [7]:
from processes.extract.functions import map_db_tables

In [8]:
db_tables = map_db_tables(engine=engine_bronze, schema=schema)

2024-12-03 19:45:38,226 - INFO - Mapping database tables in schema 'bronze' started.
2024-12-03 19:45:38,247 - INFO - Tables in schema 'bronze' were successfully mapped.


In [9]:
from model.table_relations import related_orig_bron
from datetime import date, timedelta

In [10]:
engine = engine_bronze

In [23]:
dataframes = {}

for key in related_orig_bron.keys():
    #if key in {'master_table', 'master_table_mult', 'fact_table', 'fact_table_mult'}:
    for table in related_orig_bron[key]:
        print(table['tbl_bron'])
        table_name = table['tbl_bron']
        date_column = table['date_column']

        

        try:
            

            if date_column:

                # get date range
                current_date = date.today()
                min_date = current_date - timedelta(days=730)

                current_date = current_date.strftime('%Y-%m-%d')
                min_date = min_date.strftime('%Y-%m-%d')

                query = f'SELECT * FROM "{table_name}" WHERE "{date_column}" BETWEEN \'{min_date}\' AND \'{current_date}\''

            else:
                query = f'SELECT * FROM "{table_name}"'


            dataframes[table_name] = pd.read_sql(query, con=engine).iloc[:, :-2]

            dp.logger.info(f'Data from {table_name} table has been load in a DataFrame.')

        except Exception as e:
            dp.logger.error(f"Error loading data from table '{table_name}': {e}")
            raise


2024-12-03 19:54:42,696 - INFO - Data from U6321303_Almacenes table has been load in a DataFrame.


U6321303_Almacenes
U6301303_Clientes


2024-12-03 19:54:43,142 - INFO - Data from U6301303_Clientes table has been load in a DataFrame.
2024-12-03 19:54:43,145 - INFO - Data from U6311303_Empresas table has been load in a DataFrame.
2024-12-03 19:54:43,149 - INFO - Data from U6331303_Operarios table has been load in a DataFrame.
2024-12-03 19:54:43,151 - INFO - Data from U6311303_Talleres table has been load in a DataFrame.
2024-12-03 19:54:43,153 - INFO - Data from ULSTTHPT_TiposHoras table has been load in a DataFrame.
2024-12-03 19:54:43,155 - INFO - Data from ULSTTVPT_TiposVentas table has been load in a DataFrame.


U6311303_Empresas
U6331303_Operarios
U6311303_Talleres
ULSTTHPT_TiposHoras
ULSTTVPT_TiposVentas
U6341303_Vehiculos


2024-12-03 19:54:43,930 - INFO - Data from U6341303_Vehiculos table has been load in a DataFrame.


...

U553_Compras


2024-12-03 19:54:50,230 - INFO - Data from U553_Compras table has been load in a DataFrame.


U555_Invertidas


2024-12-03 19:54:50,488 - INFO - Data from U555_Invertidas table has been load in a DataFrame.
2024-12-03 19:54:50,492 - INFO - Data from U552_Stock table has been load in a DataFrame.


U552_Stock


In [12]:
def db_tables_to_df_resent (engine, tables, date_column=None):
    ''''''
    dataframes = {}

    for key in related_orig_bron.keys():
        #if key in {'master_table', 'master_table_mult', 'fact_table', 'fact_table_mult'}:
        for table in related_orig_bron[key]:
            print(table['tbl_bron'])
            table_name = table['tbl_bron']
            date_column = table['date_column']

            try:
                if date_column:
                    # get date range
                    current_date = date.today()
                    min_date = current_date - timedelta(days=730)

                    current_date = current_date.strftime('%Y-%m-%d')
                    min_date = min_date.strftime('%Y-%m-%d')

                    query = f'SELECT * FROM "{table_name}" WHERE "{date_column}" BETWEEN \'{min_date}\' AND \'{current_date}\''

                else:
                    query = f'SELECT * FROM "{table_name}"'


                dataframes[table_name] = pd.read_sql(query, con=engine).iloc[:, :-2]

                dp.logger.info(f'Data from {table_name} table has been load in a DataFrame.')

            except Exception as e:
                dp.logger.error(f"Error loading data from table '{table_name}': {e}")
                raise
    
    return dataframes